# Preparation


Check README.md file for install/setup instructions 

**References**


https://dvc.org/doc/tutorial/define-ml-pipeline - used as example

## Initialize DVC

In [1]:
!dvc init -f

Adding '.dvc/state' to '.dvc/.gitignore'.
Adding '.dvc/lock' to '.dvc/.gitignore'.
Adding '.dvc/config.local' to '.dvc/.gitignore'.
Adding '.dvc/updater' to '.dvc/.gitignore'.
Adding '.dvc/updater.lock' to '.dvc/.gitignore'.
Adding '.dvc/state-journal' to '.dvc/.gitignore'.
Adding '.dvc/state-wal' to '.dvc/.gitignore'.
Adding '.dvc/cache' to '.dvc/.gitignore'.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|              https://dvc.org/doc/user-guide/analytics               |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: https://dvc.org/doc
- Get help and

In [2]:
%%bash

git add .
git commit -m "Initialize DVC"

[dvc-tutorial d07f5ce] Initialize DVC
 1 file changed, 484 insertions(+), 237 deletions(-)


### Files and Directories 

In [3]:
!ls -a .dvc 


.            ..           .gitignore   cache        config       updater.lock


In [4]:
!cat .dvc/.gitignore

/state
/lock
/config.local
/updater
/updater.lock
/state-journal
/state-wal
/cache

# Control versions of data

In [5]:
# Get data 

!wget -P data/ https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv
!du -sh data/*

--2019-06-09 00:03:58--  https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv
Resolving raw.githubusercontent.com... 151.101.112.133
Connecting to raw.githubusercontent.com|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3716 (3.6K) [text/plain]
Saving to: ‘data/iris.csv’

iris.csv            100%[===================>]   3.63K  --.-KB/s    in 0s      

2019-06-09 00:03:58 (50.6 MB/s) - ‘data/iris.csv’ saved [3716/3716]

4.0K	data/iris.csv


In [6]:
# Look on data

import pandas as pd

df = pd.read_csv('data/iris.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## Add flle under DVC control

In [7]:
%%bash

dvc add data/iris.csv
du -sh data/*

Adding 'data/iris.csv' to 'data/.gitignore'.
Saving 'data/iris.csv' to '.dvc/cache/57/fce90c81521889c736445f058c4838'.
Saving information to 'data/iris.csv.dvc'.

To track the changes with git run:

	git add data/.gitignore data/iris.csv.dvc
4.0K	data/iris.csv
4.0K	data/iris.csv.dvc


In [8]:
!git status -s data/

?? data/.gitignore
?? data/iris.csv.dvc


In [9]:
%%bash

git add .
git commit -m "Add a source dataset"

[dvc-tutorial 2b51aad] Add a source dataset
 2 files changed, 9 insertions(+)
 create mode 100644 data/.gitignore
 create mode 100644 data/iris.csv.dvc


## What is DVC-file?

Data file internals


>    If you take a look at the DVC-file, you will see that only outputs are defined in outs. 
    In this file, only one output is defined. The output contains the data file path in the repository and md5 cache.
    This md5 cache determines a location of the actual content file in DVC cache directory .dvc/cache
    >> Output from DVC-files defines the relationship between the data file path in a repository and the path in a cache directory. See also DVC File Format



(c) dvc.org https://dvc.org/doc/tutorial/define-ml-pipeline

In [10]:
!cat data/iris.csv.dvc

md5: 1cff89878034249db68ba6046d5b49a9
wdir: ..
outs:
- md5: 57fce90c81521889c736445f058c4838
  path: data/iris.csv
  cache: true
  metric: false
  persist: false


In [11]:
!du -sh .dvc/cache/*/*

4.0K	.dvc/cache/57/fce90c81521889c736445f058c4838


# Create ML pipeline

Stages 
- extract features 
- split dataset 
- train 
- evaluate 


## Add feature extraction stage

In [12]:
!dvc run -f stage_feature_extraction.dvc \
    -d src/featurization.py \
    -d data/iris.csv \
    -o data/iris_featurized.csv \
    python src/featurization.py

Running command:
	python src/featurization.py
Adding 'data/iris_featurized.csv' to 'data/.gitignore'.
Saving 'data/iris_featurized.csv' to '.dvc/cache/04/ed69383af337e9dabf934cbc8abc11'.
Saving information to 'stage_feature_extraction.dvc'.

To track the changes with git run:

	git add data/.gitignore stage_feature_extraction.dvc


In [13]:
!ls 

README.md                    src
data                         stage_feature_extraction.dvc
requirements.txt             tutorial.ipynb


In [15]:
!cat stage_feature_extraction.dvc

md5: 7fa7e84ef809dc5478ba9d9291915c71
cmd: python src/featurization.py
wdir: .
deps:
- md5: 33729a6a870be74dc3bc2983284a22de
  path: src/featurization.py
- md5: 57fce90c81521889c736445f058c4838
  path: data/iris.csv
outs:
- md5: 04ed69383af337e9dabf934cbc8abc11
  path: data/iris_featurized.csv
  cache: true
  metric: false
  persist: false


In [14]:
import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [15]:
!git status -s

 M data/.gitignore
?? stage_feature_extraction.dvc


In [16]:
%%bash
git add .
git commit -m "Add stage_features_extraction"

[dvc-tutorial c871bca] Add stage_features_extraction
 2 files changed, 16 insertions(+), 1 deletion(-)
 create mode 100644 stage_feature_extraction.dvc


## Add split train/test stage

In [17]:
!dvc run -f stage_split_dataset.dvc \
    -d src/split_dataset.py \
    -d data/iris_featurized.csv \
    -o data/train.csv \
    -o data/test.csv \
    python src/split_dataset.py 0.4

Running command:
	python src/split_dataset.py 0.4
Adding 'data/train.csv' to 'data/.gitignore'.
Adding 'data/test.csv' to 'data/.gitignore'.
Saving 'data/train.csv' to '.dvc/cache/3b/7d94d2e2675fb9132c42b1da980794'.
Saving 'data/test.csv' to '.dvc/cache/b3/7f00bfd2ce46c61fe744d6d26b41d3'.
Saving information to 'stage_split_dataset.dvc'.

To track the changes with git run:

	git add data/.gitignore data/.gitignore stage_split_dataset.dvc


In [18]:
!cat stage_split_dataset.dvc

md5: b9a2a7297561e2e1ac5f502eba932072
cmd: python src/split_dataset.py 0.4
wdir: .
deps:
- md5: e111aa0fa66588bf06c5f716d11bcff5
  path: src/split_dataset.py
- md5: 04ed69383af337e9dabf934cbc8abc11
  path: data/iris_featurized.csv
outs:
- md5: 3b7d94d2e2675fb9132c42b1da980794
  path: data/train.csv
  cache: true
  metric: false
  persist: false
- md5: b37f00bfd2ce46c61fe744d6d26b41d3
  path: data/test.csv
  cache: true
  metric: false
  persist: false


## Add train stage

In [19]:
!dvc run -f stage_train.dvc \
    -d src/train.py \
    -d data/train.csv \
    -o data/model.joblib \
    python src/train.py

Running command:
	python src/train.py
Adding 'data/model.joblib' to 'data/.gitignore'.
Saving 'data/model.joblib' to '.dvc/cache/d6/31150e260de432a811bfe7dd8935ec'.
Saving information to 'stage_train.dvc'.

To track the changes with git run:

	git add data/.gitignore stage_train.dvc


In [20]:
!cat stage_train.dvc

md5: 1409d57473a56f42048a84c797dbbec5
cmd: python src/train.py
wdir: .
deps:
- md5: 025acbe1552887fab33f5314d036e907
  path: src/train.py
- md5: 3b7d94d2e2675fb9132c42b1da980794
  path: data/train.csv
outs:
- md5: d631150e260de432a811bfe7dd8935ec
  path: data/model.joblib
  cache: true
  metric: false
  persist: false


### Add evaluate stage

In [22]:
!dvc run -f stage_evaluate.dvc \
    -d src/train.py \
    -d src/evaluate.py \
    -d data/test.csv \
    -d data/model.joblib \
    -m data/eval.txt \
    python src/evaluate.py

Stage is cached, skipping.


In [23]:
!cat stage_evaluate.dvc

md5: 2c5f02b139310b839b97f2a093b802b9
cmd: python src/evaluate.py
wdir: .
deps:
- md5: 025acbe1552887fab33f5314d036e907
  path: src/train.py
- md5: 9b394d26e9427759256195b47917028b
  path: src/evaluate.py
- md5: b37f00bfd2ce46c61fe744d6d26b41d3
  path: data/test.csv
- md5: d631150e260de432a811bfe7dd8935ec
  path: data/model.joblib
outs:
- md5: 1f7764d988d8d251dc3e9b1c5419f58b
  path: data/eval.txt
  cache: true
  metric: true
  persist: false


# Metrics tracking

In [24]:
!dvc metrics show

	data/eval.txt: {"f1_score": 0.7861833464670345, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 8, 0], [0, 11, 18]]}}


## Commit dvc pipelines

In [25]:
!git status -s

 M data/.gitignore
 M tutorial.ipynb
?? stage_evaluate.dvc
?? stage_split_dataset.dvc
?? stage_train.dvc


In [26]:
%%bash
git add .
git commit -m "Add pipelines"

[dvc-tutorial 6b7e36c] Add pipelines
 5 files changed, 192 insertions(+), 158 deletions(-)
 create mode 100644 stage_evaluate.dvc
 create mode 100644 stage_split_dataset.dvc
 create mode 100644 stage_train.dvc


# Reproducibility

## How does it work?

> The most exciting part of DVC is reproducibility.
>> Reproducibility is the time you are getting benefits out of DVC instead of spending time defining the ML pipelines.

> DVC tracks all the dependencies, which helps you iterate on ML models faster without thinking what was affected by your last change.
>> In order to track all the dependencies, DVC finds and reads ALL the DVC-files in a repository and builds a dependency graph (DAG) based on these files.

> This is one of the differences between DVC reproducibility and traditional Makefile-like build automation tools (Make, Maven, Ant, Rakefile etc). It was designed in such a way to localize specification of DAG nodes.
If you run repro on any created DVC-file from our repository, nothing happens because nothing was changed in the defined pipeline.

(c) dvc.org https://dvc.org/doc/tutorial/reproducibility

In [27]:
# Nothing to reproduce
!dvc repro stage_evaluate.dvc

Stage 'data/iris.csv.dvc' didn't change.
Stage 'stage_feature_extraction.dvc' didn't change.
Stage 'stage_split_dataset.dvc' didn't change.
Stage 'stage_train.dvc' didn't change.
Stage 'stage_evaluate.dvc' didn't change.
Pipeline is up to date. Nothing to reproduce.


## Add features



### Create new experiment branch

Before editing the code/featurization.py file, please create and checkout a new branch __ratio_features__

In [28]:
# create new branch

! git checkout -b ratio_features

Switched to a new branch 'ratio_features'


In [29]:
!git branch

  dvc-tutorial
  master
* ratio_features


### Update featurization.py

in file __featurization.py__ uncomment lines 

    features['sepal_length_to_sepal_width'] = features['sepal_length'] / features['sepal_width']
    features['petal_length_to_petal_width'] = features['petal_length'] / features['petal_width']

## Reproduce pipeline 

In [32]:
!dvc repro stage_evaluate.dvc

Stage 'data/iris.csv.dvc' didn't change.
Reproducing 'stage_feature_extraction.dvc'
Running command:
	python src/featurization.py
Saving 'data/iris_featurized.csv' to '.dvc/cache/cd/9e208c0232da2fb80b4c927da35dbb'.
Saving information to 'stage_feature_extraction.dvc'.
Reproducing 'stage_split_dataset.dvc'
Running command:
	python src/split_dataset.py 0.4
Saving 'data/train.csv' to '.dvc/cache/87/43ef62798f623fbaae4401f4aab654'.
Saving 'data/test.csv' to '.dvc/cache/3d/40f0c85187dda2cd9bf58b3e916630'.
Saving information to 'stage_split_dataset.dvc'.
Reproducing 'stage_train.dvc'
Running command:
	python src/train.py
Saving 'data/model.joblib' to '.dvc/cache/d7/bb60e8f731671aa212cf137c6f1e52'.
Saving information to 'stage_train.dvc'.
Reproducing 'stage_evaluate.dvc'
Running command:
	python src/evaluate.py
Saving 'data/eval.txt' to '.dvc/cache/ef/d3a0fee43c80da52d807308c56843b'.
Saving information to 'stage_evaluate.dvc'.

To track the changes with git run:

	git add stage_feature_extrac

In [33]:
# Check features used in this pipeline

import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length_to_sepal_width,petal_length_to_petal_width
0,5.1,3.5,1.4,0.2,setosa,1.457143,7.0
1,4.9,3.0,1.4,0.2,setosa,1.633333,7.0
2,4.7,3.2,1.3,0.2,setosa,1.468750,6.5
3,4.6,3.1,1.5,0.2,setosa,1.483871,7.5
4,5.0,3.6,1.4,0.2,setosa,1.388889,7.0


## Compare metrics for all runs (experiments)

In [34]:
# this pipeline metrics 

!dvc metrics show

	data/eval.txt: {"f1_score": 0.8084886128364389, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 9, 0], [0, 10, 18]]}}


In [35]:
# show all commited pipelines metrics 

!dvc metrics show -a

Working Tree:
	data/eval.txt: {"f1_score": 0.8084886128364389, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 9, 0], [0, 10, 18]]}}
dvc-tutorial:
	data/eval.txt: {"f1_score": 0.7861833464670345, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 8, 0], [0, 11, 18]]}}
ratio_features:
	data/eval.txt: {"f1_score": 0.7861833464670345, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 8, 0], [0, 11, 18]]}}


## Commit new results

In [36]:
!git status -s

 M src/featurization.py
 M stage_evaluate.dvc
 M stage_feature_extraction.dvc
 M stage_split_dataset.dvc
 M stage_train.dvc
 M tutorial.ipynb
?? src/.ipynb_checkpoints/


In [38]:
!git add .
!git commit -m "New features experiment"

On branch ratio_features
nothing to commit, working tree clean


# Checkout (start over new experiment)

- in case new features doesn't result improvements 
- or we want to improve the model by changing the hyperparameters (with OLD dataset)

## Checkout code and data files 

In [41]:
%%bash

git checkout dvc-tutorial
dvc checkout

[##############################] 100% Checkout finished!


Switched to branch 'dvc-tutorial'


In [46]:
!git branch

* dvc-tutorial
  master
  ratio_features


In [47]:
!dvc metrics show

	data/eval.txt: {"f1_score": 0.7861833464670345, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 8, 0], [0, 11, 18]]}}


In [49]:
# Nothing to reproduce since code was checked out by `git checkout`
# and data files were checked out by `dvc checkout`
!dvc repro stage_evaluate.dvc

Stage 'data/iris.csv.dvc' didn't change.
Stage 'stage_feature_extraction.dvc' didn't change.
Stage 'stage_split_dataset.dvc' didn't change.
Stage 'stage_train.dvc' didn't change.
Stage 'stage_evaluate.dvc' didn't change.
Pipeline is up to date. Nothing to reproduce.


In [51]:
!dvc metrics show

	data/eval.txt: {"f1_score": 0.7861833464670345, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 8, 0], [0, 11, 18]]}}


In [52]:
# Check features used in this pipeline

import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## Tune the model

In [53]:
# create new branch for experiment

!git checkout -b tuning
!git branch

M	tutorial.ipynb
Switched to a new branch 'tuning'
  dvc-tutorial
  master
  ratio_features
* tuning


### Change parameters of classifier (LogisticRegression)

in file __train.py__ in constructor of LogisticRegression:

* change C param to 0.1

in the end you should get such constructor of classifier:

```python
clf = LogisticRegression(C=0.1, solver='newton-cg', multi_class='multinomial', max_iter=100)
```

### Reproduce pipelines

In [54]:
# re-run pipeline 

!dvc repro stage_evaluate.dvc 

Stage 'data/iris.csv.dvc' didn't change.
Stage 'stage_feature_extraction.dvc' didn't change.
Stage 'stage_split_dataset.dvc' didn't change.
Reproducing 'stage_train.dvc'
Running command:
	python src/train.py
Saving 'data/model.joblib' to '.dvc/cache/be/122f7e293f77f4f636345868dd57c2'.
Saving information to 'stage_train.dvc'.
Reproducing 'stage_evaluate.dvc'
Running command:
	python src/evaluate.py
Saving 'data/eval.txt' to '.dvc/cache/04/a34aa507199d5f1f5c73a841a76463'.
Saving information to 'stage_evaluate.dvc'.

To track the changes with git run:

	git add stage_train.dvc stage_evaluate.dvc


In [55]:
!cat data/eval.txt

{"f1_score": 0.9639376218323586, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 17, 0], [0, 2, 18]]}}

In [56]:
!dvc metrics show -a

Working Tree:
	data/eval.txt: {"f1_score": 0.9639376218323586, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 17, 0], [0, 2, 18]]}}
dvc-tutorial:
	data/eval.txt: {"f1_score": 0.7861833464670345, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 8, 0], [0, 11, 18]]}}
ratio_features:
	data/eval.txt: {"f1_score": 0.8084886128364389, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 9, 0], [0, 10, 18]]}}
tuning:
	data/eval.txt: {"f1_score": 0.7861833464670345, "confusion_matrix": {"classes": ["setosa", "versicolor", "virginica"], "matrix": [[23, 0, 0], [0, 8, 0], [0, 11, 18]]}}


In [57]:
# Check features used in this pipeline

import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### Commit

In [58]:
%%bash

git add .
git commit -m "Tune model. C=0.1"

[tuning d9fb5a0] Tune model. C=0.1
 6 files changed, 159 insertions(+), 102 deletions(-)
 create mode 100644 src/.ipynb_checkpoints/featurization-checkpoint.py
 create mode 100644 src/.ipynb_checkpoints/train-checkpoint.py


### Merge the model to dvc-tutorial

In [59]:
%%bash

git checkout dvc-tutorial
git merge tuning

Updating 6b7e36c..d9fb5a0
Fast-forward
 src/.ipynb_checkpoints/featurization-checkpoint.py |  20 ++
 src/.ipynb_checkpoints/train-checkpoint.py         |  19 ++
 src/train.py                                       |   2 +-
 stage_evaluate.dvc                                 |   8 +-
 stage_train.dvc                                    |   6 +-
 tutorial.ipynb                                     | 206 +++++++++++----------
 6 files changed, 159 insertions(+), 102 deletions(-)
 create mode 100644 src/.ipynb_checkpoints/featurization-checkpoint.py
 create mode 100644 src/.ipynb_checkpoints/train-checkpoint.py


Switched to branch 'dvc-tutorial'


# Share data

## Setup remote storage (i.e. cloud)

In [68]:
# Create new remote

!dvc remote add -d local /tmp/dvc

Setting 'local' as a default remote.


In [69]:
# as you can see, .dvc/config is changed

!git status -s

 M .dvc/config
 M src/.ipynb_checkpoints/featurization-checkpoint.py
 M src/featurization.py
 M stage_evaluate.dvc
 M stage_feature_extraction.dvc
 M stage_split_dataset.dvc
 M stage_train.dvc
 M tutorial.ipynb


In [70]:
# check config file 

!cat .dvc/config

['remote "local"']
url = /tmp/dvc
[core]
remote = local


In [71]:
%%bash

git add .
git commit -m "Add remote storage"

[dvc-tutorial c187801] Add remote storage
 8 files changed, 118 insertions(+), 75 deletions(-)


## Push data to remote

In [72]:
# Push data to remote

!dvc push

Preparing to upload data to '/tmp/dvc'
Preparing to collect status from /tmp/dvc
[##############################] 100% Collecting information
[##############################] 100% Analysing status.
Everything is up to date.


## Pull date from remote

In [76]:
!dvc pull

Preparing to download data from '/tmp/dvc'
Preparing to collect status from /tmp/dvc
[##############################] 100% Collecting information
[##############################] 100% Analysing status.
[##############################] 100% Checkout finished!

Everything is up to date.
